In [9]:
import pandas as pd
import numpy as np
df=pd.read_csv("datos/MexicoCaseStudy.csv")

In [18]:
cont=0
for i in list(df.columns):
    for j in range(df.shape[0]):
        if df[i][j]==" ":
            cont+=1
print(cont) #no hay datos vacíos

0


Tener dataframes con datos vacíos puede generar varios inconvenientes. Puede hacer que una fila sea completamente inutil si, por ejemplo, el valor faltante correspondía a un nombre; o si se decide llenar los valores numéricos faltantes, los cálculos que se hagan posteriormente se verán afectados.

In [19]:
lista2=[]
for i in list(df["ADM2NAME"]):
    if "?" in i:
        if i not in lista2:
            lista2.append(i)
print(lista2) #lista con los municipios que tienen "?" en lugar de tildes

['San Luis R?o Colorado', 'General Plutarco El?as Ca', 'Ascensi?n', 'Nacozari de Garc?a', 'Benjam?n Hill', 'Bacad?huachi', 'Hu?pac', 'Carb?', 'Ray?n', 'G?mez Far?as', 'Jim?nez', 'Tem?sachi', 'Cuauht?moc', 'Bach?niva', 'Matach?', 'Y?cora', 'Villa Uni?n', 'Dr. Belisario Dom?nguez', 'Carich?', 'Satev?', 'Muleg?', 'Ch?nipas', 'San Ignacio R?o Muerto', 'Cuatroci?negas', 'L?pez', 'Mapim?', 'Guanacev?', 'Gustavo D?az Ordaz', 'Ind?', 'R?o Bravo', 'Ci?nega de Flores', 'Mar?n', 'Garc?a', 'G?mez Palacio', 'Pesquer?a', 'San Pedro Garza Garc?a', 'Cadereyta Jim?nez', 'Torre?n', 'M?ndez', 'Cuencam?', 'General Sim?n Bol?var', 'Peñ?n Blanco', 'San Juan del R?o', 'Concepci?n del Oro', 'G\x81?mez', 'R?o Grande', 'Jim?nez del Teul', 'Xicot?ncatl', 'Valpara?so', 'Ciudad del Ma?z', 'San Luis Potos?', 'Noria de ?ngeles', 'Cos?o', 'Tamu?n', 'Rinc?n de Romos', 'San Jos? de Gracia', 'Santa Mar?a de los ?ngele', 'Villa Garc?a', 'Pabell?n de Arteaga', 'Ru?z', 'El Plateado de Joaqu?n Am', 'Jesús Mar?a', 'Aquism?n'

In [20]:
lista1=[]
for i in list(df["ADM1NAME"]):
    if "?" in i:
        if i not in lista1:
            lista1.append(i)
print(lista1) #lista con los estados que tienen "?" en lugar de tildes 

['Nuevo Le?n', 'San Luis Potos?', 'Quer?taro de Arteaga', 'M?xico']


Columnas eliminadas: 

las columnas MUNIG_ y MUNIG_ID se supone que son de un código propio de México para nombrar estados y municipios que no tengo idea de como funciona. Además ya tengo los nombres de esos estados y municipalidades, y es mejor trabajar con ellos pues es la forma en que la gente mejor reconoce esos lugares.

las columnas ADM1CODE y ADM2CODE también son código para nombrar estados y municipios, y aunque este si entiendo como funciona, no es relevante por la misma razón que mensioné anteriormente.

Las columnas MEANILIT15, MEANSELE15, MEANCELE15, MEANSSEC15 y MEANCSEC15 fueron muy particulares. Según la información que hay sobre el dataset, esas columnas son de "número de miembros del hogar mayores de 15 años que...", números enteros pensaría uno, pero los datos estaban dados como 0,45651 por ejemplo. Como no era posible determinar a qué correspondían realmente esos datos, consideré mas apropiado no usarlos y eliminar las columnas.

La columna PERIMETER corresponde al perímetro de los municipios, pero no se mensiona en qué están dados esos perímetros (m, km, millas), como los números eran tan pequeños no era fácil deducir las unidades y no encontré información en internet al respecto. Por todo esto decidí que no era información útil

In [21]:
df=df.replace("Nuevo Le?n","Nuevo León")
df=df.replace("San Luis Potos?","San Luis Potosí")
df=df.replace("Quer?taro de Arteaga","Querétaro de Arteaga")
df=df.replace("M?xico","México")

df=df.drop(["MUNIG_","MUNIG_ID","ADM1CODE","ADM2CODE","PERIMETER","MEANILIT15","MEANSELE15","MEANCELE15","MEANSSEC15","MEANCSEC15"],axis=1)

NuevosNombresColumnas={"ADM1NAME": "ESTADOS", 
                        "ADM2NAME": "MUNICIPIOS",
                        "RURPER":"POBLACION RURAL",
                        "FGT_0RUR00":"POBREZA ALIMENTARIA 2000",
                        "FGT_0RUR02":"POBREZA ALIMENTARIA 2002",
                        "ACCESO":"ACCESO (min)"
                      } 
df=df.rename(columns=NuevosNombresColumnas)
df=df.drop_duplicates(["MUNICIPIOS"], keep='last', ignore_index=True) #tenía varias filas repetidas, por ello elimino los municipios repetidos.
df=df.sort_values("ESTADOS",ignore_index=True)
df.to_csv("datos/MexicoPobrezaOrdenado.csv", index=False)

arregloEstados=df["ESTADOS"].unique()
arregloEstados=np.sort(arregloEstados)
indices=np.arange(0,arregloEstados.shape[0])
Estados=pd.DataFrame(arregloEstados,columns=["ESTADO"],index=indices)#dataframe de una columna con los estados

print("Buscando municipios mas apartados...")
dfMasApartados=pd.DataFrame(columns=["ESTADOS","MUNICIPIOS","ACCESO (min)"])
for i in range(Estados.shape[0]):
    estado=Estados["ESTADO"][i]
    dfPorEstado1=df.loc[df["ESTADOS"]==estado]
    MasApartados=dfPorEstado1.sort_values("ACCESO (min)", ascending=False, ignore_index=True)[0:3]
    MasApartados=MasApartados.loc[:,["ESTADOS","MUNICIPIOS","ACCESO (min)"]]
    dfMasApartados=pd.concat([dfMasApartados, MasApartados],axis=0, ignore_index=True)
dfMasApartados.to_csv("datos/MunicipiosMasApartados.csv", index=False)
print("finalizado")

print("Buscando promedio de población rural...")
dfRURAL=pd.DataFrame(columns=["ESTADOS","POBLACION RURAL (MEDIA)"])
for i in range(Estados.shape[0]):
    estado=Estados["ESTADO"][i]
    dfPorEstado2=df.loc[df["ESTADOS"]==estado]
    mediaRural=dfPorEstado2["POBLACION RURAL"].mean()
    fila=pd.DataFrame([[Estados["ESTADO"][i],mediaRural]],columns=dfRURAL.columns)
    dfRURAL=dfRURAL.append(fila, ignore_index=True)
dfRURAL=dfRURAL.sort_values("POBLACION RURAL (MEDIA)", ascending=False,ignore_index=True)
dfRURAL.to_csv("datos/MayorPoblacionRural.csv", index=False)
print("finalizado")


#POBREZA ALIMENTARIA 2002 y POBREZA ALIMENTARIA 2000 son la fracción prevista de hogares rurales por debajo del umbral de pobreza alimentaria en 2002 y 2000
#a continuación las convierto en porcentaje y busco la diferencia
print("iniciando filtrado según pobreza alimentaria")
dfDIFERENCIA=pd.DataFrame(columns=["ESTADOS","MUNICIPIOS","POBREZA ALIMENTARIA 2000","POBREZA ALIMENTARIA 2002","DIFERENCIA"])
filaDiferencia=[]
for i in range(df.shape[0]):
    df["POBREZA ALIMENTARIA 2000"][i]*=100
    df["POBREZA ALIMENTARIA 2002"][i]*=100
    diferencia=df["POBREZA ALIMENTARIA 2000"][i]-df["POBREZA ALIMENTARIA 2002"][i]
    filaDiferencia.append(diferencia)
df["DIFERENCIA"]=filaDiferencia
for i in range (Estados.shape[0]):
    estado=Estados["ESTADO"][i]
    dfPorEstado3=df.loc[df["ESTADOS"]==estado]
    dfPorEstado3=dfPorEstado3.sort_values("DIFERENCIA", ascending=False, ignore_index=True)
    nuevaFila1=dfPorEstado3.iloc[[0]]
    nuevaFila1=nuevaFila1.loc[:,["ESTADOS","MUNICIPIOS","POBREZA ALIMENTARIA 2000","POBREZA ALIMENTARIA 2002","DIFERENCIA"]]
    nuevaFila2=dfPorEstado3.iloc[[-1]] 
    nuevaFila2=nuevaFila2.loc[:,["ESTADOS","MUNICIPIOS","POBREZA ALIMENTARIA 2000","POBREZA ALIMENTARIA 2002","DIFERENCIA"]]
    dfDIFERENCIA=pd.concat([dfDIFERENCIA, nuevaFila1],axis=0, ignore_index=True)
    dfDIFERENCIA=pd.concat([dfDIFERENCIA, nuevaFila2],axis=0, ignore_index=True)
dfDIFERENCIA.to_csv("datos/PobrezaAlimentaria.csv", index=False)    
print("finalizado")

TypeError: drop_duplicates() got an unexpected keyword argument 'ignore_index'

Municipios mas apartados:
Me pareció importante tomar los tres municipios de cada estado mas apartados de los centros urbanos pues pueden ser los mas propensos a quedar olvidados y a que no les llegue recursos.

Población rural:
Esto no muestra en sí la media de porcentaje población rural porque hace falta considerar otros factores. Lo que realmente quería era mostrar qué estados tienen mayor cantidad de municipios con alto porcentaje de población rural, porque creo que estos estados tienen un reto aun mas grande para superar el resago, en cuanto a oportunidades, presente en las zonas rurales.

Pobreza alimenticia:
Con esto busco resaltar, por cada estado, el que mas lograró disminuir el porcentaje de hogares por debajo del umbral de pobreza alimenticia y el municipio que menos lo disminuyó o que incluso lo aumentó. Hago esto pues algunas de las consecencias mas frecuentes de la pobreza extrema son el hambre y la desnutrición, entonces uno así puede hacer un contraste entre los que mejoraron y los que empeoraron para posteriormente mirar en esos lugares lo que se ha hecho y lo que se puede mejorar. En sí yo no encuentro esas respuestas pues no tengo esos datos, sino que indico los lugares en los que se debe mirar.

In [16]:
from matplotlib import pyplot as plt 
import random

def mostrarMunicipiosApartados(dato):
    municipios=dfMasApartados.loc[dfMasApartados["ESTADOS"]==dato]
    municipios=municipios.drop(["ESTADOS"],axis=1)
    municipios=municipios.reset_index(drop=True)
    print("los 3 municipios de {} mas alejados de los centros urbanos son:".format(dato))
    print(" ")
    print(municipios)
    print(" ")
    
    municipios.plot.bar(x="MUNICIPIOS", y="ACCESO (min)", rot=0)
    plt.show()
    
def mostrarPoblacionRural(dato):
    print("el promedio del porcentaje de población rural en {} es:".format(dato))
    print(" ")
    indice=(list(dfRURAL["ESTADOS"])).index(dato)
    print(dfRURAL.iloc[[indice]])
    print(" ")
    
    municipios=df.loc[df["ESTADOS"]==dato]
    municipios=municipios.reset_index(drop=True)
    municipios=municipios.loc[:,["MUNICIPIOS","POBLACION RURAL"]]
    
    plt.ion()
    listaColores=['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
    color=random.choice(listaColores)              
    x=np.arange(0,municipios["MUNICIPIOS"].shape[0])
    y=np.array(municipios["POBLACION RURAL"])
    
    plt.scatter(x, y, s=100, c=color, marker='o',alpha=0.9)
    plt.title("Porcentaje de poblacion rural de sus municipios")
    plt.ylabel("%")
    plt.show()
    
def mostrarPobrezaAlimenticia(dato):
    municipios=dfDIFERENCIA.loc[dfDIFERENCIA["ESTADOS"]==dato]
    municipios=municipios.drop(["ESTADOS"])
    dfParaGraficar=municipios.drop(["DIFERENCIA"])
    dfParaGrafica=dfParaGrafica.set_index("MUNICIPIOS") 
    print("Se muestra el municipio de {} que mayor cambio tuvo y el que menor cambio tuvo entre 2000 y 2002 \n".format(dato))
    print("Nota: una diferencia negativa indica que AUMENTÓ el porcentaje de hogares por debajo del umbral pobreza alimenticia de 2000 a 2002 \n")
    print(municipios)
    
    dfParaGraficar.plot.bar(rot=0)
    plt.show()
    
    
    

In [ ]:
arregloMunicipios=df["MUNICIPIOS"]
arregloMunicipios=np.sort(arregloMunicipios)
indices=np.arange(0,arregloMunicipios.shape[0])
Municipios=pd.DataFrame(arregloMunicipios,columns=["MUNICIPIO"],index=indices)#dataframe de una columna con los municipios

While True:
    
    